In [2]:
import torch

In [3]:
from datasets import load_dataset

In [4]:
train_dataset = load_dataset("jfleg", split='validation[:]')

Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


In [5]:
eval_dataset = load_dataset("jfleg", split='test[:]')

Using custom data configuration default
Reusing dataset jfleg (/home/ubuntu/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b)


In [6]:
for case in train_dataset["corrections"][5:9]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

['They draw in the consumers , like me , to purchase a great product with amazing ingredients , but actually , they just make up these words to increase their sales . ', 'They draw in consumers like me , who purchase this great product with all these amazing ingredients , but sometimes they just make things up to increase their sales . ', 'They draw the consumers , like me , to purchase this great product with all these amazing features , but actually sometimes they just make something up just to increase their sales . ', 'They draw the consumers , like me , to purchase this great product with all these amazing ingredients and all that , but actually they just sometimes make something up just to increase their sales . ']
They draw in the consumers , like me , to purchase a great product with amazing ingredients , but actually , they just make up these words to increase their sales . 
--------------------------------------------------------
['I want to talk about nocive or bad products 

In [7]:
import csv

In [32]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases are blank strings. So we'll skip them
                #input_text = remove_excess_spaces(input_text)
                correction = remove_excess_spaces(correction)
                input_text = remove_excess_spaces(input_text)


                if input_text and correction:
                    writter.writerow([input_text, correction])

In [9]:
from pathlib import Path

In [10]:
data_dir = Path.cwd()/'datasets/jfleg'

In [33]:
generate_csv("jfleg_train.csv", train_dataset)

In [34]:
generate_csv("jfleg_validation.csv", eval_dataset)

In [12]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text


In [14]:
import pandas as pd

In [20]:
df = pd.read_csv('jfleg_train.csv', index_col=0, header=None)

In [22]:
df = df.iloc[1: , :]  # drop first row which has titles

In [17]:
z = df.to_dict()

In [23]:
df.head

<bound method NDFrame.head of                                                                                                     1
0                                                                                                    
grammar: So I think we can not live if old peop...  So I think we would not be alive if our ancest...
grammar: So I think we can not live if old peop...  So I think we could not live if older people d...
grammar: So I think we can not live if old peop...  So I think we can not live if old people could...
grammar: So I think we can not live if old peop...  So I think we can not live if old people can n...
grammar: For not use car .                                                  Not for use with a car . 
...                                                                                               ...
grammar: Sceene of violence can affect on them .                Scenes of violence can affect them . 
grammar: While the communities in general have ...  

In [24]:
z = df.to_dict()

In [26]:
type(z)

dict

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [31]:

text = "I am lived with my parenmts "
print(f"untrained input: {text}")
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')
print(f"inputes = {inputs}")
output = model.generate(inputs['input_ids'], num_beams=4, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
print("".join(correction)) #Correction: I am living with my parents.


untrained input: I am lived with my parenmts 
Ignored unknown kwarg option direction
inputes = {'input_ids': tensor([[19519,    10,   196,   183,  4114,    28,    82,   260,    35,    51,
            17,     7,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Grammatik:I am lived with my parenmts with my parenmts.
